In [89]:
import torch
import torch.nn as nn 
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split
from xgboost.sklearn import XGBClassifier


In [90]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',', dtype=np.float32)
    x = data[:, 0:-2]
    y = data[:, [-1]] # after 3 months (-2) and 6 months (-1)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [91]:
# target to fix code


class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Sigmoid())
        self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        self.dropout = nn.Dropout(0.0001)
        #self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Tanh())
        self.activation = torch.nn.ELU()
        #self.dropout = nn.Dropout(0.1)
        #self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        #self.dropout = nn.Dropout(0.1)
        #self.activation = torch.nn.ReLU()
        
        
        

        
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.activation(x)
        
        
        return x # return model

In [92]:
x_train, y_train = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_training_dataset.csv')
x_test, y_test = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_test_dataset.csv')

In [93]:
y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
      

In [94]:
x_test

tensor([[0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.2000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.3000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 0.7000, 0.6111, 1.0000, 0.1652, 0.4800],
        [0.0042, 0.4179, 0.2535, 0.5000, 0.2500, 0.2727, 0.3986, 0.0000, 0.0000,
         0.2388, 0.6000, 0.5000, 1.0000, 0.1739, 0.7500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 1.0000, 0.7222, 0.0000, 0.1739, 0.0000],
        [0.3208, 0.4467, 0.2676, 0.5000, 0.7500, 0.2727, 0.7489, 0.2812, 0.3559,
         0.1939, 1.0000, 0.7222, 0.0000, 0.3043, 0.7500],
        [0.0084, 0.5389, 0.0000, 1.0000, 0.2500, 0.1818, 0.6514, 0.0000, 0.0735,
         0.3152, 0.4000, 1.0000, 1.0000, 0.2609, 0.0000],
        [0.0168, 0.7176, 0.

In [95]:
x_train.shape

torch.Size([103, 15])

In [96]:


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # test set에는 transform만 사용하기


x_train=torch.Tensor(x_train)
x_test=torch.Tensor(x_test)

In [97]:
model = FNN(input_size=15, hidden_size=300, output_size=1)
criterion = nn.BCELoss()
#optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix
optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix

In [98]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.1178, -0.0677, -0.1561,  ..., -0.2026, -0.1694,  0.2099],
        [-0.1176, -0.0067,  0.0407,  ...,  0.0366, -0.1200,  0.1745],
        [-0.0216,  0.0263, -0.0932,  ...,  0.0057,  0.0605,  0.2395],
        ...,
        [ 0.1885, -0.2320, -0.1747,  ..., -0.0592, -0.0775,  0.0297],
        [-0.0610,  0.0397, -0.2311,  ..., -0.1441,  0.0605,  0.1597],
        [ 0.0726, -0.2264, -0.2460,  ...,  0.1159, -0.1016, -0.1015]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0979,  0.0743,  0.0702,  0.1679, -0.0329,  0.1755, -0.0787,  0.0296,
        -0.1690,  0.1220,  0.1237, -0.1948, -0.2120, -0.1344, -0.1658, -0.0055,
        -0.2485, -0.1433, -0.0372,  0.0151,  0.2294,  0.1663, -0.1689, -0.1981,
        -0.0568, -0.0986, -0.1893, -0.1574,  0.1323,  0.2227,  0.2448,  0.0758,
        -0.0777, -0.0324, -0.2375,  0.0123,  0.0603, -0.1889, -0.1812, -0.0127,
         0.2289,  0.0268, -0.2145, -0.1582, -0.1728, -0.0426, -0.0764, -0.0005,
         0.0589

In [99]:
# assuming y is torch.Tensor

In [100]:
num_epochs = 1001

for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
        

with torch.no_grad(): 
    train_accuracy = (model(x_train).gt(0.5).float() == y_train).float().mean().item()
    test_accuracy = (model(x_test).gt(0.5).float() == y_test).float().mean().item()
    
    

print(f'\nTrain Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
#Train Accuracy: 0.9709 -> start
#Test Accuracy: 0.9000 -> start

Epoch [0/1001], Loss: 0.7433
Epoch [200/1001], Loss: 0.1370


Epoch [400/1001], Loss: 0.0534
Epoch [600/1001], Loss: 0.0319
Epoch [800/1001], Loss: 0.0241
Epoch [1000/1001], Loss: 0.0196

Train Accuracy: 1.0000
Test Accuracy: 0.9500


In [101]:
model(x_test).gt(0.5)

tensor([[False],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False]])

In [102]:
model(x_test).gt(0.5).float()

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [103]:
y_test

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.]])